In [3]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import torch
import spacy
import json
import numpy as np
import matplotlib.pyplot as plt

/Users/moritzblum/miniconda3_m1/envs/blp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Derive relation features
This script derives different kinds of features from the relations description.


## Description Embeddings

Load data and language model for embedding

In [9]:
import torch

model = SentenceTransformer('all-mpnet-base-v2')

text_file = './data/Wikidata5M/relation2text.txt'

relation_uri_2_id = torch.load('./data/Wikidata5M/maps.pt')['rel_ids']
id_2_relation_uri = {i: r for r, i in relation_uri_2_id.items()}

Derivation of description embeddings with the language model. The embeddings are stored according to the relation ids (relation_uri_to_id) in the matrix x.

In [12]:
x = torch.zeros((len(relation_uri_2_id.keys()), 768))

with open(text_file) as descriptions_file_in:
    for i, line in enumerate(tqdm(descriptions_file_in.readlines())):
        line = line[:-1]
        uri, description = line.split('\t', 1)




        embeddings = model.encode(description)
        try:
            x[relation_uri_2_id[uri]] = torch.tensor(embeddings)
        except KeyError:
            print(uri)
            continue


  7%|▋         | 58/825 [00:02<00:39, 19.65it/s]

P1012


 26%|██▋       | 218/825 [00:11<00:30, 19.65it/s]

P301


 40%|███▉      | 329/825 [00:16<00:24, 19.94it/s]

...

 55%|█████▍    | 451/825 [00:23<00:18, 19.79it/s]

P1204


 96%|█████████▌| 790/825 [00:40<00:01, 20.42it/s]

P423


100%|██████████| 825/825 [00:42<00:00, 19.64it/s]


In [13]:
torch.save(x, './data/Wikidata5M/relation_description_embedding.pt')

In [15]:
x.size()

torch.Size([822, 768])